In [1]:
import pandas as pd
import numpy as np

In [2]:
df1 = pd.read_csv("311_Service_Requests_2022.csv")
df2 = pd.read_csv("census_data2.csv")

In [3]:
df1 = df1.dropna(subset=['zip_code'])

In [4]:
# df1['zip_code'] = df1['zip_code'].astype(int)
# df2['zip_code'] = df2['zip_code'].astype(int)

In [5]:
print(df1.shape)
print(df2.shape)

(333674, 26)
(33774, 50)


In [6]:
# print(df1.head(2))
# print(df2.head(2))

In [7]:
df = pd.merge(df1, df2, on='zip_code', how='left', suffixes=('_left', '_right'))

In [8]:
print(df.shape)

(333674, 75)


In [9]:
# Replace -666666666 with an empty string
df = df.replace(-666666666, '')
df = df.replace(-888888888, '')

In [10]:
print(df.columns)

Index(['ticket_id', 'issue_type', 'issue_description', 'case_owner',
       'case_owner_description', 'street_address', 'city', 'state', 'zip_code',
       'neighborhood_district', 'created_year_month',
       'ticket_created_date_time', 'ticket__last_update_date_time',
       'ticket_closed_date_time', 'ticket_status', 'location_city',
       'sr_xcoordinate', 'sr_ycoordinate', 'latitude', 'longitude',
       'method_received', 'sr_priority', 'goal_days', 'actual_completed_days',
       'ticket_year', 'ObjectId', 'NAME', 'DP02_0079E', 'DP02_0079PE',
       'DP02_0095E', 'DP02_0095PE', 'DP02_0097E', 'DP02_0097PE', 'DP02_0114E',
       'DP02_0114PE', 'DP02PR_0001E', 'DP02PR_0001PE', 'DP03_0045E',
       'DP03_0045PE', 'DP03_0051E', 'DP03_0051PE', 'DP03_0062E', 'DP03_0062PE',
       'DP03_0068E', 'DP03_0068PE', 'DP03_0078E', 'DP03_0078PE', 'DP03_0088E',
       'DP03_0088PE', 'DP03_0119E', 'DP03_0119PE', 'DP04_0001E', 'DP04_0001PE',
       'DP04_0003E', 'DP04_0003PE', 'DP04_0005E', 'DP04_

In [11]:
print(df.shape)

(333674, 75)


In [12]:
df = df.rename(columns={
    'DP03_0013E': 'fml_inlaborf',
    'DP03_0013PE': 'fml_inlaborf_pr',
    'DP03_0011E':'fml_emp',
    'DP03_0011PE':'fml_emp_pr',
    'DP03_0009E':'unemployment',
    'DP03_0009PE':'unemployment_pr',
    'DP05_0070E':'latino',
    'DP05_0070PE':'latino_pr',
    'DP05_0038E':'aa',
    'DP05_0038PE':'aa_pr',
    'DP05_0037E':'white',
    'DP05_0037PE':'white_pr',
    'DP05_0001E':'total_pop',
    'DP05_0001PE':'total_pop_pr',
    'DP04_0134E': 'median_rent',
    'DP04_0134PE': 'median_rent_pr',
    'DP04_0047E':'occupied_hunits_rent',
    'DP04_0047PE':'occupied_hunits_rent_pr',
    'DP04_0005E': 'hunits_rental',
    'DP04_0005PE': 'hunits_rental_pr',
    'DP04_0003E':'hunits_vacant',
    'DP04_0003PE':'hunits_vacant_pr',
    'DP04_0001E':'hunits_total',
    'DP04_0001PE':'hunits_total_pr',
    'DP03_0119E':'poverty',
    'DP03_0119PE':'poverty_pr',
    'DP03_0088E': 'inc_percapita',
    'DP03_0088PE': 'inc_percapita_pr',
    'DP03_0078E': 'inc_15_25',
    'DP03_0078PE': 'inc_15_25_pr',
    'DP02_0079E': 'pop_1year&over', 
    'DP02_0079PE': 'pop_1year&over_pr',
    'DP02_0095E' : 'foreign_pop',
    'DP02_0095PE':'foreign_pop_pr',
    'DP02_0097E' : 'not_citizen',
    'DP02_0097PE' : 'not_citizen_pr',
    "DP02_0114E":'lang_spoken',
    'DP02_0114PE': "lang_spoken_pr",
    'DP02PR_0001E': "total_hh",
    'DP02PR_0001PE': "total_hh_pr",
    'DP03_0045E': "civil_emp",
    'DP03_0045PE': "civil_emp_pr",
    'DP03_0051E' :"inc_total",
    'DP03_0051PE' :"inc_total_pr",
    'DP03_0062E' : "inc_median",
    'DP03_0062PE' : "inc_median_pr",
    'DP03_0068E' : 'inc_retirement',
    'DP03_0068PE' : 'inc_retirement_pr'
    })

In [13]:
df['created_year_month']=df['created_year_month'].astype(str)
df['month'] = df['created_year_month'].str.slice(start=4)

In [14]:
def month_to_text(month_number):
    months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
              'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    return months[int(month_number) - 1]

In [15]:
df['month'] = df['month'].apply(month_to_text)

In [16]:
df['intime'] = np.where(df['goal_days'] - df['actual_completed_days'] >= 0, 1, 0)

In [17]:
print(df.shape)

(333674, 77)


In [18]:
df.to_csv('combined_data.csv', index=False)

In [19]:
import math
df = df[df['total_pop'] != 0]
df['median_rent']=pd.to_numeric(df['median_rent'])
df['latino_pr'] = df['latino']/df['total_pop']
df['log_pop'] = df['total_pop'].apply(lambda x: math.log(x))
df['log_hunits']= df['hunits_total'].apply(lambda x: math.log(x))
df['log_percapita']=df['inc_percapita'].apply(lambda x: math.log(x))
df['log_foreign_pop']=df['foreign_pop'].apply(lambda x: math.log(x))
df['log_inc_total']=df['inc_total'].apply(lambda x: math.log(x))
df['log_median_rent']=df['median_rent'].apply(lambda x: math.log(x))
df['log_inc_median']=df['inc_median'].apply(lambda x: math.log(x))


In [20]:
df = df.drop(['ticket_id','issue_description','case_owner_description', 'street_address','state', 'location_city',
              'neighborhood_district','created_year_month', 'ticket_created_date_time','ticket__last_update_date_time',
              'ticket_closed_date_time','ticket_year','ticket_status','sr_xcoordinate','sr_ycoordinate','actual_completed_days',
              'ticket_year','ObjectId','NAME','pop_1year&over','pop_1year&over_pr','foreign_pop_pr','not_citizen',
              'fml_inlaborf','unemployment','poverty','lang_spoken','total_hh','total_hh_pr','hunits_total_pr','civil_emp','inc_total_pr',
              'inc_median_pr','inc_retirement','inc_15_25','inc_percapita_pr','poverty','hunits_total_pr','hunits_vacant',
              'hunits_rental_pr','occupied_hunits_rent','median_rent_pr','total_pop_pr','white','white_pr','aa','latino_pr',
              'fml_emp','fml_inlaborf',
              'latino','total_pop','hunits_total','inc_percapita','foreign_pop','inc_total','median_rent','inc_median','log_hunits'], axis=1)

In [21]:
print(df.shape)

(333669, 30)


In [22]:
df = df.dropna()

In [23]:
df.to_csv('processed_data.csv', index=False)

In [24]:
print(df.columns)

Index(['issue_type', 'case_owner', 'city', 'zip_code', 'latitude', 'longitude',
       'method_received', 'sr_priority', 'goal_days', 'not_citizen_pr',
       'lang_spoken_pr', 'civil_emp_pr', 'inc_retirement_pr', 'inc_15_25_pr',
       'poverty_pr', 'hunits_vacant_pr', 'hunits_rental',
       'occupied_hunits_rent_pr', 'aa_pr', 'unemployment_pr', 'fml_emp_pr',
       'fml_inlaborf_pr', 'month', 'intime', 'log_pop', 'log_percapita',
       'log_foreign_pop', 'log_inc_total', 'log_median_rent',
       'log_inc_median'],
      dtype='object')


In [25]:
print(df.shape)

(331455, 30)
